# Laden der Daten
In diesem Notebook werden die Daten eingelesen, welche direkt über die URL zugängig sind. Die Geodaten für das Weg- und Velonetz <br>
in Zürich müssen manuell auf der Open Data Seite runtergeladen und dann über einen CSV-Import geladen werden, nachdem die Datei Entzipped wurde.<br>
Grund hierfür ist die Auswahl des Ausschnitts, in welchem das Velonetz angezeigt werden soll. <br>
Dieser muss auf der Seite ausgewählt werden und es wird dann ein Downlaodlink für 5 Minuten zur Verfügung gestellt.


## Verwendete Links/Quellen

In [1]:
url1 = "https://www.hev-schweiz.ch/vermieten/nebenkostenabrechnungen/sonnenscheindauer"
url2 = "https://www.hev-schweiz.ch/vermieten/statistiken/mietpreise/durchschnittliche-mietpreise"
url3 = "https://opendata.swiss/de/dataset/median-strompreis-per-kanton"

# Installation notwendiger Bibliotheken

In [2]:
!pip install --upgrade pip
!pip install beautifulsoup4
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


# Module importieren

In [3]:
from bs4 import BeautifulSoup as Soup
import urllib.request
import openpyxl
import pandas as pd
from pathlib import Path

# Funktionen

In [50]:
def tableDataText(table):
    def rowgetDataText(tr, coltag='td'):
        return [td.get_text(strip=True) for td in tr.find_all(coltag)]
    rows = []
    trs = table.find_all('tr')
    headerow = rowgetDataText(trs[0], 'th')
    if headerow:
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs:
        rows.append(rowgetDataText(tr, 'td') )
    return rows

In [108]:
html = urllib.request.urlopen(url1).read()
soup = Soup(html, parser='lxml')
htmltables = soup.find_all('table', { 'class' : 'contenttable acc_content ce-table' })

all_lists = []
for table in htmltables:
  all_lists.append(tableDataText(table))

all_dataframes = []
for list in all_lists:
  df = pd.DataFrame(list)
  all_dataframes.append(df)

print(all_dataframes)

year = 2024
for dataframe in all_dataframes:
  filepath = Path('/content/Sonnenscheindauer' + str(year) + '.xlsx')
  filepath.parent.mkdir(parents=True, exist_ok=True)
  dataframe.to_excel(filepath, index=False, header=False)
  year = year - 1


[                      0    1    2    3    4    5    6    7    8    9    10  \
0                Station  Jan  Feb  Mär  Apr  Mai  Jun  Jul  Aug  Sep  Okt   
1              Adelboden   75   97   84  112   99  116  185  213  106        
2                  Aigle   73  102  111  125  118  150  244  256  129        
3                Altdorf   46   73   95  115  103  123  173  190  120        
4        Basel-Binningen   63   56  110  121  153  153  225  285  128        
5      Bern / Zollikofen   61   90  108  143  151  160  239  273  131        
6            Buchs-Aarau   53   73   90  126  138  146  213  264  112        
7      La Chaux-de-Fonds   91   78  112  120  122  141  225  267  126        
8                   Chur   93   79  104  135  132  131  216  238  119        
9                Cimetta  165  128  146  193   98  145  258  299  126        
10                 Davos  111   96  121  144  113  129  198  216  130        
11     Disentis / Sedrun   85   63   79  122   97  123  189  23